<a href="https://colab.research.google.com/github/DataEngineering-Amber/Collabs-and-Assignments/blob/main/Completed_9_29_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flask App for Downloading Data from JSONPlaceholder into ORM Models

In this notebook, we will build a Flask app to download data from `jsonplaceholder.typicode.com`, store it in a database using SQLAlchemy ORM, and test the ORM models and relationships.

We will create the following tables:
- Users
- Posts
- Comments
- Albums
- Photos
- Todos

Let's get started!

## 1. Flask Setup with SQLAlchemy ORM

We will use Flask as the web framework and SQLAlchemy as the ORM to manage the database. First, let's initialize our app and configure the database.

In [1]:
# Install necessary packages
!pip install Flask SQLAlchemy requests pytest Flask-SQLAlchemy

In [2]:
# Flask app setup with SQLAlchemy ORM
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///jsonplaceholder.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

# Define the ORM models

In [3]:
# User model
class User(db.Model):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(80))
    username = db.Column(db.String(80))
    email = db.Column(db.String(120))
    address = db.Column(db.String(200))
    phone = db.Column(db.String(20))
    website = db.Column(db.String(100))
    company = db.Column(db.String(100))

    posts = db.relationship('Post', backref='user', lazy=True)
    albums = db.relationship('Album', backref='user', lazy=True)
    todos = db.relationship('Todo', backref='user', lazy=True)


In [4]:
# Post model
class Post(db.Model):
    __tablename__ = 'posts'
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('users.id'), nullable=False)
    title = db.Column(db.String(200))
    body = db.Column(db.Text)

    comments = db.relationship('Comment', backref='post', lazy=True)


In [5]:
# Comment model
class Comment(db.Model):
    __tablename__ = 'comments'
    id = db.Column(db.Integer, primary_key=True)
    post_id = db.Column(db.Integer, db.ForeignKey('posts.id'), nullable=False)
    name = db.Column(db.String(80))
    email = db.Column(db.String(120))
    body = db.Column(db.Text)


In [6]:
# Album model
class Album(db.Model):
    __tablename__ = 'albums'
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('users.id'), nullable=False)
    title = db.Column(db.String(200))

    photos = db.relationship('Photo', backref='album', lazy=True)


In [7]:
# Photo model
class Photo(db.Model):
    __tablename__ = 'photos'
    id = db.Column(db.Integer, primary_key=True)
    album_id = db.Column(db.Integer, db.ForeignKey('albums.id'), nullable=False)
    title = db.Column(db.String(200))
    url = db.Column(db.String(300))
    thumbnail_url = db.Column(db.String(300))


In [8]:
# Todo model
class Todo(db.Model):
    __tablename__ = 'todos'
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('users.id'), nullable=False)
    title = db.Column(db.String(200))
    completed = db.Column(db.Boolean)


## 2. Downloading Data from JSONPlaceholder

Now, let's write a function to download data from `jsonplaceholder.typicode.com` and insert it into our database.

In [9]:
# Function to download and insert data from JSONPlaceholder
import requests

with app.app_context():
  db.create_all()

  def download_data():
      # Download users
      users = requests.get('https://jsonplaceholder.typicode.com/users').json()
      for user_data in users:
          user = User(id=user_data['id'], name=user_data['name'], username=user_data['username'],
                    email=user_data['email'], address=str(user_data['address']), phone=user_data['phone'],
                    website=user_data['website'], company=str(user_data['company']))
          db.session.add(user)
      db.session.commit()

      # Download posts
      posts = requests.get('https://jsonplaceholder.typicode.com/posts').json()
      for post_data in posts:
          post = Post(id=post_data['id'], user_id=post_data['userId'], title=post_data['title'],
                      body=post_data['body'])
          db.session.add(post)
      db.session.commit()

      # Download comments
      comments = requests.get('https://jsonplaceholder.typicode.com/comments').json()
      for comment_data in comments:
          comment = Comment(id=comment_data['id'], post_id=comment_data['postId'],
                        name=comment_data['name'], email=comment_data['email'], body=comment_data['body'])
          db.session.add(comment)
      db.session.commit()

      # Download albums, photos, and todos similarly
  download_data()

## 3. Pytest for ORM Models and Relationships

Let's now add unit tests using `pytest` to ensure our models and their relationships are correct.

In [10]:
# Pytest setup
import pytest

@pytest.fixture(scope='module')
def test_client():
    app.config['TESTING'] = True
    with app.test_client() as testing_client:
        with app.app_context():
            db.create_all()
            yield testing_client
            db.drop_all()


In [11]:
with app.app_context():
    # Test for models and relationships
    def test_user_creation(test_client):
        user = User(name='Test User', username='testuser',
                    email='test@example.com', address='Test Address', phone='1234567890',
                    website='https://example.com', company='Test Company')
        db.session.add(user)
        db.session.commit()
        assert user in db.session

    def test_post_creation(test_client):
        user = User.query.first()
        post = Post(user_id=user.id, title='Test Post', body='Test Body')
        db.session.add(post)
        db.session.commit()
        assert post in db.session

    def test_comment_creation(test_client):
        post = Post.query.first()
        comment = Comment(post_id=post.id, name='Test Comment',
                          email='comment@example.com', body='Comment Body')
        db.session.add(comment)
        db.session.commit()
        assert comment in db.session

## 4. Exercises: Querying Data and Relationships Using SQLAlchemy ORM with Unit Tests

In these exercises, you will practice querying data and relationships using the SQLAlchemy ORM and writing unit tests using `pytest`. Each exercise focuses on specific relationships between models like Users, Posts, Comments, Albums, and Todos.

Complete each exercise by writing the necessary query and writing a corresponding unit test.

### Exercise 1: Query All Users

Write a query to retrieve all users from the `users` table and display their names and emails.

**Unit Test**: Write a unit test that checks if at least one user exists in the database.

In [12]:
with app.app_context():
    # Query for all users
    users = User.query.all()
    for user in users:
        print(user.name, user.email)

    # Unit test example
    def test_query_all_users(test_client):
        users = User.query.all()
        assert len(users) > 0

Leanne Graham Sincere@april.biz
Ervin Howell Shanna@melissa.tv
Clementine Bauch Nathan@yesenia.net
Patricia Lebsack Julianne.OConner@kory.org
Chelsey Dietrich Lucio_Hettinger@annie.ca
Mrs. Dennis Schulist Karley_Dach@jasper.info
Kurtis Weissnat Telly.Hoeger@billy.biz
Nicholas Runolfsdottir V Sherwood@rosamond.me
Glenna Reichert Chaim_McDermott@dana.io
Clementina DuBuque Rey.Padberg@karina.biz


### Exercise 2: Get Posts by a Specific User

Write a query to retrieve all posts by a user with a specific username (e.g., `Bret`). Print the titles of the posts.

**Unit Test**: Write a unit test that checks if the correct number of posts is returned for the user.

In [13]:
with app.app_context():
    # Query for posts by a specific user


    # Unit test example
    def test_query_user_posts(test_client):
     # write the test here

SyntaxError: incomplete input (<ipython-input-13-b7d2b5ad81cf>, line 7)

In [15]:
with app.app_context():
    # Query posts by user with username 'Bret'
    user = User.query.filter_by(username='Bret').first()
    if user:
        posts = Post.query.filter_by(user_id=user.id).all()
        for post in posts:
            print(post.title)
    else:
        print("User not found")


sunt aut facere repellat provident occaecati excepturi optio reprehenderit
qui est esse
ea molestias quasi exercitationem repellat qui ipsa sit aut
eum et est occaecati
nesciunt quas odio
dolorem eum magni eos aperiam quia
magnam facilis autem
dolorem dolore est ipsam
nesciunt iure omnis dolorem tempora et accusantium
optio molestias id quia eum


In [16]:
def test_query_user_posts(test_client):
    with app.app_context():
        user = User.query.filter_by(username='Bret').first()
        assert user is not None  # Make sure the user exists

        posts = Post.query.filter_by(user_id=user.id).all()
        assert len(posts) > 0  # Check that user has at least one post

### Exercise 3: Get Comments on a Specific Post

Write a query to retrieve all comments for a post with a specific title (e.g., `qui est esse`). Print the names and emails of the commenters.

**Unit Test**: Write a unit test that checks if the comments are correctly associated with the post.

In [ ]:
with app.app_context():
  # Query for comments on a specific post


  # Unit test example
  def test_query_post_comments(test_client):
      #write the test here

et fugit eligendi deleniti quidem qui sint nihil autem Presley.Mueller@myrl.com
repellat consequatur praesentium vel minus molestias voluptatum Dallas@ole.me
et omnis dolorem Mallory_Kunze@marie.org
provident id voluptas Meghan_Littel@rene.us
eaque et deleniti atque tenetur ut quo ut Carmen_Keeling@caroline.name


In [17]:
with app.app_context():
    # Query for comments on the post titled 'qui est esse'
    post = Post.query.filter_by(title='qui est esse').first()
    if post:
        comments = Comment.query.filter_by(post_id=post.id).all()
        for comment in comments:
            print(comment.name, comment.email)
    else:
        print("Post not found")


et fugit eligendi deleniti quidem qui sint nihil autem Presley.Mueller@myrl.com
repellat consequatur praesentium vel minus molestias voluptatum Dallas@ole.me
et omnis dolorem Mallory_Kunze@marie.org
provident id voluptas Meghan_Littel@rene.us
eaque et deleniti atque tenetur ut quo ut Carmen_Keeling@caroline.name


In [18]:
def test_query_post_comments(test_client):
    with app.app_context():
        post = Post.query.filter_by(title='qui est esse').first()
        assert post is not None  # Make sure the post exists

        comments = Comment.query.filter_by(post_id=post.id).all()
        assert len(comments) > 0  # Check that post has at least one comment


In [21]:
with app.app_context():
    db.session.remove()  # Properly closes the session
    db.engine.dispose()  # Closes the database connection
    print("Database connection closed successfully!")


Database connection closed successfully!
